In [1]:
import pandas as pd
import numpy as np
import functools
from sklearn import tree, model_selection
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import gc
import pickle
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree, export_text
import sys
from sklearn.tree import _tree

In [2]:
class colors:
    '''Colors class:reset all colors with colors.reset; two
    sub classes fg for foreground
    and bg for background; use as colors.subclass.colorname.
    i.e. colors.fg.red or colors.bg.greenalso, the generic bold, disable,
    underline, reverse, strike through,
    and invisible work with the main class i.e. colors.bold'''
    reset = '\033[0m'
    bold = '\033[01m'
    disable = '\033[02m'
    underline = '\033[04m'
    reverse = '\033[07m'
    strikethrough = '\033[09m'
    invisible = '\033[08m'

    class fg:
        black = '\033[30m'
        red = '\033[31m'
        green = '\033[32m'
        orange = '\033[33m'
        blue = '\033[34m'
        purple = '\033[35m'
        cyan = '\033[36m'
        lightgrey = '\033[37m'
        darkgrey = '\033[90m'
        lightred = '\033[91m'
        lightgreen = '\033[92m'
        yellow = '\033[93m'
        lightblue = '\033[94m'
        pink = '\033[95m'
        lightcyan = '\033[96m'

    class bg:
        black = '\033[40m'
        red = '\033[41m'
        green = '\033[42m'
        orange = '\033[43m'
        blue = '\033[44m'
        purple = '\033[45m'
        cyan = '\033[46m'
        lightgrey = '\033[47m'

#print(colors.bg.green, "SKk", colors.fg.red, "Amartya")
#print(colors.bg.lightgrey, "SKk", colors.fg.red, "Amartya")

sys.setrecursionlimit(5000)
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        if class_names is None:
            rule += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
        rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]
        
    return rules

def uniform_prior(prior):
    result = prior.copy()
    spread = 1./len(prior)
    for k in prior.keys():
        result[k] = spread
    return result

# function to fix the prior prob of PASS class, the rest of the classes shall have uniform prob.
def fix_PASS_prior(prior, value):
    result = prior.copy()
    remain = (1 - value)/(len(prior)-1)
    for k in prior.keys():
        if k == "PASS":
            result[k] = value
        else:
            result[k] = remain
    return result

def encode_column(coldata, index):
    encoded = []
    for r in coldata:
        # check for nan by doing this comparison.
        if r != r:
            encoded.append([])
        else:
            # first filter the invalid codes from the list.
            valid_codes = list(filter(lambda x: x in index, str(r).split(",")))
            # map the codes to the index values.
            codes = list(map(lambda x: index[x], valid_codes)) #str(r).split(",")))
            encoded.append(codes)
    return encoded

def transform_denial_code(c, denial_index):
    if c != c:
        return 0
    return denial_index[c]

In [3]:
data = pd.read_excel(r'/home/sudarsun/notebooks/Sudar_sir_claims_scrubber_data_historical_v1.xlsx')

In [4]:
data.columns

Index(['claim_id', 'patient_id', 'payer_id', 'plan_name', 'denial_code',
       'code_activity', 'activity_desc', 'type_activity', 'act_type_dsc',
       'pdx', 'sdx', 'Reason_for_visit', 'consolidated_diagnoses'],
      dtype='object')

In [5]:
nrecords = len(data)

In [6]:
# handle NAs in the denial code outputs
data["denial_code"] = data["denial_code"].fillna("PASS")
dc_distrib = data["denial_code"].value_counts()
print(dc_distrib)

PASS         478574
MNEC-004     102590
PRCE-002       7419
PRCE-001       7305
CODE-010       4061
MNEC-005       3894
CLAI-012       1827
NCOV-0026      1518
DUPL-002       1413
PRCE-006        609
COPY-001        400
NCOV-001        337
AUTH-001        308
PRCE-010        286
CODE-014        245
ELIG-006        216
NCOV-003        213
ELIG-001        210
PRCE-007        186
TIME-001        178
CLAI-008        117
ELIG-007        116
AUTH-003        105
CLAI-016         62
BENX-002         59
BENX-005         57
AUTH-005         50
ELIG-005         43
MNEC-003         42
AUTH-004         29
DUPL-001          2
CLAI-018          1
Name: denial_code, dtype: int64


In [7]:
denial_codes = pd.unique(data["denial_code"])
print("labels found", denial_codes)

labels found ['PASS' 'MNEC-004' 'PRCE-010' 'PRCE-002' 'CLAI-012' 'MNEC-005' 'CODE-010'
 'DUPL-002' 'PRCE-001' 'PRCE-006' 'CODE-014' 'AUTH-001' 'AUTH-003'
 'CLAI-016' 'PRCE-007' 'NCOV-003' 'NCOV-001' 'AUTH-005' 'ELIG-005'
 'ELIG-001' 'ELIG-007' 'ELIG-006' 'CLAI-008' 'MNEC-003' 'AUTH-004'
 'NCOV-0026' 'TIME-001' 'BENX-002' 'COPY-001' 'BENX-005' 'CLAI-018'
 'DUPL-001']


### Set the required denial codes of interest

In [8]:
# set the classes of interest.
#denial_codes = list(dc_distrib.keys())
denial_codes = ["PASS", "MNEC-004"]
print('class labels of interest', denial_codes)

class labels of interest ['PASS', 'MNEC-004']


### filter the cpt codes by some constraints

In [56]:
# now filter the CPT codes that are starting with "8"
selected_cpts = []
for code in data["code_activity"]:
    if str(code)[0] == '8':
        selected_cpts.append(code)

In [16]:
# we are strongly assuming that the data is not multi-label.
# get the unique cpt codes
#cpt = pd.unique(list(data["code_activity"]))
cpt = pd.unique(selected_cpts)
print("unique CPT codes ->", cpt)
print("# unique CPTs: ", len(cpt))

unique CPT codes -> [84481 86376 84443 84439 80061 84100 82248 80053 85025 83525 82533 82565
 82310 80051 82947 82024 82040 84450 83036 84460 84520 85610 85730 83202
 81001 87510 87660 87480 82607 82728 84630 84550 86431 86140 82306 83540
 83735 83970 82785 86003 86703 86900 86901 86803 87340 85060 84436 82247
 83690 84295 84132 80076 82465 84702 84478 84484 82043 85652 84153 85379
 84466 82746 82330 83520 82308 86800 84432 89322 87804 82378 86301 80047
 82150 82272 83993 87169 87338 87425 82962 84075 88142 82670 83001 83002
 84146 87086 83550 82550 83021 86592 86762 80069 82105 84305 84403 84156
 86225 86038 82977 86160 82274 87230 82805 87807 83615 87880 82374 83090
 86147 85300 85303 86039 86060 86200 86850 82627 82960 82955 86304 84144
 84270 84402 83498 86885 86778 86777 86644 86645 83516 87809 87899 87040
 87045 86235 84154 86000 87220 87270 87081 87210 87205 87481 84480 86300
 87070 87076 86141 83013 83014 82340 88720 87177 87329 82784 85045 84155
 82553 85660 82435 87801 87075 

In [17]:
#clear previously owned
if "pdx_icds" in locals():
    # release the memory blocks
    del(pdx_icds)
    del(sdx_icds)
    del(rov_icds)
    gc.collect()
    
# collect the icd codes from the diagnosis columns
pdx_icds = [str(code).split(',') for code in data[data["pdx"].notna()]["pdx"]]
sdx_icds = [str(code).split(',') for code in data[data["sdx"].notna()]["sdx"]] 
rov_icds = [str(code).split(',') for code in data[data["Reason_for_visit"].notna()]["Reason_for_visit"]]

In [18]:
# collect the pdx items
pdx = []
for row in pdx_icds:
    for code in row:
        pdx.append(code)
# collect the sdx items
sdx = []
for row in sdx_icds:
    for code in row:
        sdx.append(code)
# collect the rov items        
rov = []
for row in rov_icds:
    for code in row:
        rov.append(code)

### set the ICD min support

In [19]:
# identify the codes that does not have the min-support
icd_minsupport = 5

# compute the ICD distribution
icd_distrib = pd.DataFrame(pdx + sdx + rov).value_counts()

# now filter the ICD codes that don't have the required support.
icd_to_retain = np.array(list(filter(lambda x: icd_distrib[x] >= icd_minsupport, icd_distrib.keys()))).flatten()

print("unique icd codes ->", icd_to_retain)
print("# unique icds: ", len(icd_to_retain))

unique icd codes -> ['E55.9' 'R53.83' 'D50.9' ... 'H01.131' 'T23.212A' 'S56.424D']
# unique icds:  5337


In [20]:
# forward and reverse index the unique codes.
icd_indices = dict((i,c) for c,i in enumerate(icd_to_retain))
cpt_indices = dict((str(c),i) for i, c in enumerate(cpt))
denial_indices = dict((str(c),i) for i, c in enumerate(denial_codes))

In [21]:
print(denial_indices)
print(denial_codes)

{'PASS': 0, 'MNEC-004': 1}
['PASS', 'MNEC-004']


### Set the type activity

In [22]:
type_activity = 3

### Also filter by CPT codes

In [36]:
# construct a new data frame with only the necessary data.
dc_array = denial_codes.copy()
# also limit the scope of only CPT codes, type_activity=3
new_data = data[data["denial_code"] == dc_array[0]][data["type_activity"]==type_activity]
new_data = new_data[new_data["code_activity"] < 90000][new_data["code_activity"] >= 80000]
dc_array.pop(0)
for dc in dc_array:
    mydf = data[data["denial_code"] == dc][data["type_activity"]==type_activity]
    mydf = mydf[mydf["code_activity"] < 90000][mydf["code_activity"] >= 80000]
    new_data = new_data.append(mydf)
print(new_data["denial_code"].value_counts())

# update the no of records.
nrecords = len(new_data)

# @TODO we should destroy the original data frame for memory saving.

/mnt/store/userdata/sudarsun/claims/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/mnt/store/userdata/sudarsun/claims/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/mnt/store/userdata/sudarsun/claims/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/mnt/store/userdata/sudarsun/claims/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


PASS        292166
MNEC-004     92458
Name: denial_code, dtype: int64


In [37]:
# encode the claim result
#claim_status = new_data["denial_code"] == "PASS"
#let's first compute the class prior
#prior = claim_status.value_counts(normalize=True)

prior = new_data["denial_code"].value_counts(dropna=False, normalize=True)
print(prior)

PASS        0.759615
MNEC-004    0.240385
Name: denial_code, dtype: float64


### Set the training and testing priors as required

In [38]:
# set the required testing data prior
test_prior = prior
#test_prior = uniform_prior(prior)
print("\nRequired test prior:\n",test_prior)

# override the train prior if needed
#train_prior = prior
#train_prior = uniform_prior(prior)
train_prior = fix_PASS_prior(prior, 0.7)
print("\nRequired train prior:\n", train_prior)


Required test prior:
 PASS        0.759615
MNEC-004    0.240385
Name: denial_code, dtype: float64

Required train prior:
 PASS        0.7
MNEC-004    0.3
Name: denial_code, dtype: float64


### set the required training and testing corpus sizes

In [53]:
# set the number of required training & testing sample sizes
ntrain = 307000
ntest = 1000

In [54]:
available = new_data["denial_code"].value_counts()
print(available)
for dc in denial_codes:
    av = available[dc]
    tr = int(train_prior[dc]*ntrain)
    te = int(test_prior[dc]*ntest)
    print(colors.fg.blue, "class:", dc, "available=", av, "req_train=", tr, "req_test=", te)
    if (tr+te) > av:
        print(colors.bold, colors.fg.red, "\nPROBLEM: class", dc, "required", tr+te, "> available",av,"please adjust ntrain and ntest", colors.reset)
        sys.exit(1)
        

PASS        292166
MNEC-004     92458
Name: denial_code, dtype: int64
 class: PASS available= 292166 req_train= 214900 req_test= 759
 class: MNEC-004 available= 92458 req_train= 92100 req_test= 240


In [55]:
# now collect the ids for training and testing samples.
train_ids = []    
test_ids = []

cum_train = 0
cum_test = 0
# now, we have to draw the sample from tr_indexed data points, hopefully following the class prior.
for c in prior.keys():
    # get the required training data count
    req_ntrain = int(train_prior[c] * ntrain)
    req_ntest = int(test_prior[c] * ntest)
    
    # get the keys from the data
    #keys = data[data["denial_code"]==c]["denial_code"].keys()
    keys = new_data[new_data["denial_code"]==c]["denial_code"].keys()
    print(c, "--> available points:", len(keys), "|| required train points:", req_ntrain, "|| test points:", req_ntest)

    # permutate the keys.
    keys = np.random.permutation(keys)
    
    # if the required count is less than available, slice the array into train and test.
    if len(keys) > req_ntrain + req_ntest:
        train_ids = train_ids + list(keys[0:req_ntrain])
        test_ids = test_ids + list(keys[req_ntrain:(req_ntrain+req_ntest)])
    else:
        print(colors.bold, colors.fg.red, "\nPROBLEM: required samples greater than available, please adjust ntrain and ntest", colors.reset)
        break
    
    #for k in keys:
    #    if k in tr_index and len(train_ids)-cum_train < req_ntrain:
    #        train_ids.append(k)
    #    if k in te_index and len(test_ids)-cum_test < req_ntest:
    #        test_ids.append(k)
    print("got", len(train_ids)-cum_train, "train &", len(test_ids)-cum_test, "test points.")
    cum_train = len(train_ids)
    cum_test = len(test_ids)

print("\nRequired train data prior:\n", train_prior)
print("\nActual train data prior:\n", new_data["denial_code"][train_ids].value_counts(normalize=True))
print("\nRequired test data prior:\n", test_prior)
print("\nActual test data prior:\n", new_data["denial_code"][test_ids].value_counts(normalize=True))

PASS --> available points: 292166 || required train points: 214900 || test points: 759
got 214900 train & 759 test points.
MNEC-004 --> available points: 92458 || required train points: 92100 || test points: 240
got 92100 train & 240 test points.

Required train data prior:
 PASS        0.7
MNEC-004    0.3
Name: denial_code, dtype: float64

Actual train data prior:
 PASS        0.7
MNEC-004    0.3
Name: denial_code, dtype: float64

Required test data prior:
 PASS        0.759615
MNEC-004    0.240385
Name: denial_code, dtype: float64

Actual test data prior:
 PASS        0.75976
MNEC-004    0.24024
Name: denial_code, dtype: float64


In [57]:
# create the model name based on the settings.
distrib = new_data["denial_code"][train_ids].value_counts()
part1 = ",".join(map(lambda x: x + "~" + str(distrib[x]), distrib.keys()))
tr_distrib = new_data["denial_code"][train_ids].value_counts(normalize=True)
part2 = ",".join(map(lambda x: x + "~" + str(round(tr_distrib[x],2)), tr_distrib.keys()))
te_distrib = new_data["denial_code"][test_ids].value_counts(normalize=True)
part3 = ",".join(map(lambda x: x + "~" + str(round(te_distrib[x],2)), te_distrib.keys()))
model_name = "pop=" + part1 + "~~tr=" + part2 + "~~val=" + part3

model_name += "~~cpt=8XXXX"

print(model_name)

pop=PASS~214900,MNEC-004~92100~~tr=PASS~0.7,MNEC-004~0.3~~val=PASS~0.76,MNEC-004~0.24~~~cpt=8xxxx


In [58]:
#encode the PDX column
pdx_encoded_tr = encode_column(new_data["pdx"][train_ids], icd_indices)
pdx_encoded_te = encode_column(new_data["pdx"][test_ids], icd_indices)

In [59]:
#encode the SDX column
sdx_encoded_tr = encode_column(new_data["sdx"][train_ids], icd_indices)
sdx_encoded_te = encode_column(new_data["sdx"][test_ids], icd_indices)

In [60]:
# encode the ROV column
rov_encoded_tr = encode_column(new_data["Reason_for_visit"][train_ids], icd_indices)
rov_encoded_te = encode_column(new_data["Reason_for_visit"][test_ids], icd_indices)

In [61]:
# encode the CPT column
cpt_encoded_tr = encode_column(new_data["code_activity"][train_ids], cpt_indices)
cpt_encoded_te = encode_column(new_data["code_activity"][test_ids], cpt_indices)

In [62]:
# encode the denial column
denial_encoded_tr = list(map(lambda x: transform_denial_code(x, denial_indices), new_data["denial_code"][train_ids]))
denial_encoded_te = list(map(lambda x: transform_denial_code(x, denial_indices), new_data["denial_code"][test_ids]))

In [63]:
def one_hot(record, num_classes):
    encoded = np.zeros(num_classes)
    for r in record:
        encoded[r] = 1.
    return encoded

In [64]:
# free up the space explicitly
if 'X_train' in locals():
    del(X_train) 
    del(X_test)
    gc.collect()

X_train = [np.hstack((one_hot(cpt_encoded_tr[i], num_classes=len(cpt_indices)), 
                      one_hot(rov_encoded_tr[i], num_classes=len(icd_indices)),
                      one_hot(sdx_encoded_tr[i], num_classes=len(icd_indices)),
                      one_hot(pdx_encoded_tr[i], num_classes=len(icd_indices)))) for i in range(len(train_ids))]
X_test  = [np.hstack((one_hot(cpt_encoded_te[i], num_classes=len(cpt_indices)), 
                      one_hot(rov_encoded_te[i], num_classes=len(icd_indices)),
                      one_hot(sdx_encoded_te[i], num_classes=len(icd_indices)),
                      one_hot(pdx_encoded_te[i], num_classes=len(icd_indices)))) for i in range(len(test_ids))]

In [65]:
# free up the space explicitly
if 'features' in locals():
    del(features)
    gc.collect()

features = list(cpt_indices.keys()) + ["RFV-"+k for k in icd_indices.keys()] + ["SDX-"+k for k in icd_indices.keys()] + ["PDX-"+k for k in icd_indices.keys()]
print(len(features))

16481


In [66]:
model_name += "~~n_features=" + str(len(features))
model_name += "~~type-activity=" + str(type_activity)

In [68]:
model_already_available = True
#check if the file is already available, if so, load it.
try:
    with open(model_name + ".pkl", "rb") as model_fp:
        print(colors.fg.blue, "MODEL file", model_name + ".pkl", "is already available, loading it.." ,colors.reset)
        dt = pickle.load(model_fp)
        model_fp.close()
except FileNotFoundError:
    print(colors.fg.red, "MODEL file", model_name + ".pkl", "is not available, building it.." ,colors.reset)
    # if the file is not present, let's create the model freshly.
    model_already_available = False
    dt = tree.DecisionTreeClassifier(min_samples_leaf=50, class_weight="balanced")
    dt.fit(X_train, denial_encoded_tr)
    # save the model.
    with open(model_name + ".pkl", "wb") as file:
        pickle.dump(dt, file)
        file.close()

 MODEL file pop=PASS~214900,MNEC-004~92100~~tr=PASS~0.7,MNEC-004~0.3~~val=PASS~0.76,MNEC-004~0.24~~~cpt=8xxxx~~n_features=16481~~type-activity=3 is not available, building it.. 


In [69]:
# let's run the prediction only if the model is built from scratch
if model_already_available == False:
    y_pred = dt.predict(X_test)
    y_hat_train = dt.predict(X_train)

In [70]:
# let's run the prediction only if the model is built from scratch
if model_already_available == False:
    print(classification_report(denial_encoded_tr, y_hat_train, zero_division=0))
    print("Accuracy =", accuracy_score(denial_encoded_tr, y_hat_train))

              precision    recall  f1-score   support

           0       0.94      0.81      0.87    214900
           1       0.67      0.88      0.76     92100

    accuracy                           0.83    307000
   macro avg       0.80      0.85      0.82    307000
weighted avg       0.86      0.83      0.84    307000

Accuracy = 0.8334820846905537


In [71]:
# let's run the prediction only if the model is built from scratch
if model_already_available == False:
    #cm = confusion_matrix(y_test, y_pred)
    cm = confusion_matrix(denial_encoded_te, y_pred)
    #print(cm)

    print(classification_report(denial_encoded_te, y_pred, zero_division=0))
    print("Test Accuracy =", accuracy_score(denial_encoded_te, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.82      0.88       759
           1       0.61      0.88      0.72       240

    accuracy                           0.84       999
   macro avg       0.78      0.85      0.80       999
weighted avg       0.87      0.84      0.85       999

Test Accuracy = 0.8368368368368369


In [72]:
# save the model.
#with open(model_name, "wb") as file:
#    pickle.dump(dt, file)
#    file.close()

In [73]:
#rf = RandomForestClassifier(n_jobs=10)
#rf.fit(X_train, denial_encoded_tr)
#rf.fit(X_train, y_train)

In [74]:
#yhat = rf.predict(X_test)

### plot the tree.

In [75]:
#plt.figure(figsize=(100,40))
#plot_tree(dt, filled=True, rounded=True, impurity=True, feature_names=features)

### Generate the rules now

In [76]:
# extract the induced rules and save them to a file.
rules = get_rules(dt, features, denial_codes)
rule_text = "\n".join(rules)

with open(model_name + ".rules", "wt") as rule_file:
    rule_file.write(rule_text)
    rule_file.close()
    
    #for r in rules:
        #print(r)
        